<a href="https://colab.research.google.com/github/Ratnasari124/Mechine_Learning_Ratnasari/blob/main/Ratnasari_2241720007_Mesin_Learning__JS_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Job Sheet 10**

Recurrent Neural Network (RNN)


---

Nama : Ratnasari (2241720007)


---
Link Jobsheet : https://polinema.gitbook.io/jti-modul-praktikum-pembelajaran-mesin/job-sheet-10-recurrent-neural-network-rnn

Link Github :

# **PRAKTIKUM 1**

---

RNN untuk Analisis Sentimen

In [4]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [5]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

## Setup input pipeline

---



Download dataset menggunakan TFDS. Lihat loading text tutorial jika ingin me load data secara manua

In [6]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

Awalnya ini mengembalikan dataset (teks, pasangan label):

In [7]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


Berikutnya acak data untuk pelatihan dan membuat kumpulan pasangan (teks, label) ini:

In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b'Some people seem to think this was the worst movie they have ever seen, and I understand where they\'re coming from, but I really have seen worse.<br /><br />That being said, the movies that I can recall (ie the ones I haven\'t blocked out) that were worse than this, were so bad that they physically pained every sense that was involved with watching the movie. The movies that are worse than War Games 2 are the ones that make you want to gouge out your eyes, or stab sharp objects in your ears to keep yourself from having another piece of your soul ripped away from you by the awfulness.<br /><br />War Games: The Dead Code isn\'t that bad, but it comes pretty close. Yes I was a fan of the original, but no I wasn\'t expecting miracles from this one. Let\'s face it the original wasn\'t really that great of a movie in the first place, it was basically just a campy 80s teen romance flick with some geek-appeal to it.<br /><br />That\'s all I was hoping for, something bad, but that m

Buat Teks Encoder

---



In [9]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [10]:
vocab=np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [11]:
encoded_example=encoder(example)[:3].numpy()
encoded_example

array([[ 47,  83, 289, ...,   0,   0,   0],
       [142,   4, 323, ...,   0,   0,   0],
       [ 11,   7, 614, ...,   0,   0,   0]])

In [12]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'Some people seem to think this was the worst movie they have ever seen, and I understand where they\'re coming from, but I really have seen worse.<br /><br />That being said, the movies that I can recall (ie the ones I haven\'t blocked out) that were worse than this, were so bad that they physically pained every sense that was involved with watching the movie. The movies that are worse than War Games 2 are the ones that make you want to gouge out your eyes, or stab sharp objects in your ears to keep yourself from having another piece of your soul ripped away from you by the awfulness.<br /><br />War Games: The Dead Code isn\'t that bad, but it comes pretty close. Yes I was a fan of the original, but no I wasn\'t expecting miracles from this one. Let\'s face it the original wasn\'t really that great of a movie in the first place, it was basically just a campy 80s teen romance flick with some geek-appeal to it.<br /><br />That\'s all I was hoping for, something bad, but that

## Buat Model


---



In [13]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [14]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
sample_text_sensor =tf.constant([sample_text])
encoded_text = encoder(sample_text_sensor)
predictions = model.predict(encoded_text)
print(predictions[0])


Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 663s 2s/step - accuracy: 0.5179 - loss: 0.6813 - val_accuracy: 0.7547 - val_loss: 0.5813
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 661s 2s/step - accuracy: 0.8048 - loss: 0.4342 - val_accuracy: 0.8448 - val_loss: 0.3606
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 669s 2s/step - accuracy: 0.8477 - loss: 0.3502 - val_accuracy: 0.8771 - val_loss: 0.3332
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 671s 2s/step - accuracy: 0.8629 - loss: 0.3198 - val_accuracy: 0.8490 - val_loss: 0.3545
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 686s 2s/step - accuracy: 0.8634 - loss: 0.3161 - val_accuracy: 0.8375 - val_loss: 0.3311
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 687s 2s/step - accuracy: 0.8656 - loss: 0.3122 - val_accuracy: 0.8354 - val_loss: 0.3452
Epoch 7/10
197/391 ━━━━━━━━━━━━━━━━━━━━ 5:20 2s/step - accuracy: 0.8685 - loss: 0.3044

In [ ]:
# predict on a sample text with padding

padding = "the" * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

#** PRAKTIKUM 2**


---

Generator Teks dengan RNN


**Setup**

Import TensorFlow

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',  # This should be the filename you want to save it as
    origin='https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'  # This is the URL of the file
)

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Load Data

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


**Olah Teks**

Vectorize Teks


In [ ]:
example_texts=['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


**Membuat Batch Training**

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Buat Model**

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      # Fix: Initialize the state with the correct shape using tf.zeros
      batch_size = tf.shape(x)[0]
      states = tf.zeros([batch_size, self.gru.units], dtype=tf.float32) # Initialize with zeros and correct shape

    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### **Uji Model**

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 66)               │          67,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([33, 36, 58, 23, 45,  4, 52,  9, 46, 35,  7, 28, 40,  0, 14, 48, 28,
       40, 23, 56,  0, 39, 24, 52, 19, 49, 22,  9, 25, 44, 33, 42, 16, 46,
       30,  0, 17, 32, 60, 59, 57, 25, 62,  1, 49, 52, 49, 42,  8,  6, 14,
       18, 12, 63, 31, 22, 24, 33, 40, 22, 60, 11, 18,  4, 15,  9, 33, 51,
       29, 64, 14, 65, 52, 57, 34, 59,  9, 36, 43, 44, 42, 28, 55, 32, 60,
       31, 22, 21, 43, 18, 11,  5, 60, 11, 56, 30,  2, 47, 65, 26])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"friend,\nAnd I'll not wish thee to her.\n\nPETRUCHIO:\nSignior Hortensio, 'twixt such friends as we\nFew "

Next Char Predictions:
 b"TWsJf$m.gV,Oa[UNK]AiOaJq[UNK]ZKmFjI.LeTcCgQ[UNK]DSutrLw\njmjc-'AE;xRIKTaIu:E$B.TlPyAzmrUt.WdecOpSuRIHdE:&u:qQ hzM"


**Train Model**

Tambahan optimizer dan fungsi loss


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.187941, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

65.88699

In [ ]:
model.compile(optimizer='adam', loss=loss)

Konfigurasi Cekpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Add .weights.h5 to the filename
checkpoint_prefix += ".weights.h5"  # This line is the fix

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Lakukan Proses Training


In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 991s 6s/step - loss: 3.1066
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 972s 6s/step - loss: 1.9254
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1000s 6s/step - loss: 1.6340
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1009s 6s/step - loss: 1.4835
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 981s 6s/step - loss: 1.3941
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 989s 6s/step - loss: 1.3330
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 973s 6s/step - loss: 1.2792
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 984s 6s/step - loss: 1.2361
Epoch 9/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 979s 6s/step - loss: 1.1957
Epoch 10/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 974s 5s/step - loss: 1.1510
Epoch 11/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 989s 6s/step - loss: 1.1133
Epoch 12/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 979s 6s/step - loss: 1.0702
Epoch 13/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 953s 6s/step - loss: 1.0248
Epoch 14/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 981s 6s/step - loss: 0.9768
Epoch 15/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 

### **Generate Teks**

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model  # Changed model8 to model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
What a foot is that he straight?

CATESBY:
No, sir, he, and give it your grace's wife:
I' the main blatter, entreathed the crebt,
Which almost husbands, though thou wast done to thee
Than with a perpetual sicketing.

MENENIUS:
Twice fives on a charm, to be them
To coll him here alone: Lewis, if God forslike
High banishment, the rest, if this hand soul
Near to use me wear an else.

PAULINA:
Well, be was have:
'Tis very short!

PARIS:
Thy grief is Elentand's pains to her soft,
And so I have heard him straight.

LUCENTIO:
While we weep, so dear as I exproceded,
Wherein thy great good corns once as mount?
Of vain, thou wretch'd! conveyent me.

CAMILLO:
Be a think, follow, gentle sorrow!

Second Gentleman:
This is the lamb. Yet as muny as my remains:
Let me in suspicion; for a Clarence would,
Like him their lips been kiss'd upon him, sir,
I must be green, let Romeo cheers;
But now 'tisched by the Earl of Whice to-day.
Red this war I hear, the rest resign my good
Quietness, though the

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThis deed is quick hear it.\n\nHERMIONE:\nMenry in all but slipp'd, let's so tender\nWise men rage, with rooms and the Aufiditees,\nWas like an oath reward, when he detested?\n\nFirst Huntsman:\nImen these sorrows is simple and their tribble.\n\nPETRUCHIO:\nShould it not hard it?\n\nLUCIO:\n\nISABELLA:\nTell near, Gremio, rids unto the tongue.\n\nKING RICHARD III:\nThink you, my noble cousin Hark!\nThis tooth thy bridies self to visit him,\nAs an hundred cavernings his action.\nAn officer, throne; I have been out-day nor need,\nAnd not believe thee quiet creft his name;\nAh, so I firm, 'tis better sleeps with which stamps\nAbove a ballad is repeals, and cries 'God save thee;\nMuth, I think she will come retiren;\nAnd often when famous store your master!\nLorb, Isabella, some woman.\n\nLUCIO:\nI take Henry from his sones, and by the hostess of thy name!\nBesides, he wounded he will settled.\n\nPETRUCHIO:\nNow comes these are the Antilus, brings it out, as your\nfai

**Ekspor Model Generator**

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

NameError: name 'tf' is not defined

# **TUGAS**

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}